In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributedDense
from keras.layers import LSTM
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
# TBD different random seeds
np.random.seed(7)

### On statefulness

Making a RNN stateful means that the states for the samples of each batch will be reused as initial states for the samples in the next batch.

When using stateful RNNs, it is therefore assumed that:

- all batches have the same number of samples
- if X1 and X2 are successive batches of samples, then X2[i] is the follow-up sequence to X1[i], for every i.

Notes that the methods predict, fit, train_on_batch, predict_classes, etc. will all update the states of the stateful layers in a model. This allows you to do not only stateful training, but also stateful prediction.




In [3]:
# whether to use LSTM or MLP
use_LSTM = True

# number of features used in the regression (for MLP)
mlp_num_features = 10
#

# predict several timesteps at once
lstm_predict_sequences = True
lstm_num_predictions = 5

# lstm_num_timesteps
lstm_num_timesteps = 5
# lstm_num_features
lstm_num_features = 1
# stateful?
lstm_stateful = False
# use two lstm layers?
lstm_stack_layers = False

# window_size
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features

batch_size = 1
num_epochs = 200
num_neurons = 4

# scale the dataset to values between scale_min and scale_max
scale = False
scale_min = -1
scale_max = 1
#scaler = MinMaxScaler(feature_range=(scale_min, scale_max))
scaler = StandardScaler()

In [4]:
# various test datasets
ts_train_lineartrend = np.arange(1,101, dtype='float64').reshape(-1,1)  
ts_test_lineartrend_outofrange = np.arange(101,121, dtype='float64').reshape(-1,1)
ts_test_lineartrend_withinrange = np.arange(21,41, dtype='float64').reshape(-1,1)


In [5]:
testname = 'linear_trend_within_range'
ts_train = ts_train_lineartrend
ts_test = ts_test_lineartrend_withinrange
ts_all = np.append(ts_train, ts_test).reshape(-1,1)
len_overall = len(ts_all)

In [6]:
len_overall

120

In [7]:
ts_all.dtype

dtype('float64')

In [8]:
ts_train.shape, ts_test.shape

((100, 1), (20, 1))

In [9]:
ts_train[:10]

array([[  1.],
       [  2.],
       [  3.],
       [  4.],
       [  5.],
       [  6.],
       [  7.],
       [  8.],
       [  9.],
       [ 10.]])

In [10]:
ts_test[:10]

array([[ 21.],
       [ 22.],
       [ 23.],
       [ 24.],
       [ 25.],
       [ 26.],
       [ 27.],
       [ 28.],
       [ 29.],
       [ 30.]])

In [11]:
if scale:
    ts_train = scaler.fit_transform(ts_train)
    ts_test = scaler.transform(ts_test)

In [12]:
ts_train[:10]

array([[  1.],
       [  2.],
       [  3.],
       [  4.],
       [  5.],
       [  6.],
       [  7.],
       [  8.],
       [  9.],
       [ 10.]])

In [13]:
ts_test[:10]

array([[ 21.],
       [ 22.],
       [ 23.],
       [ 24.],
       [ 25.],
       [ 26.],
       [ 27.],
       [ 28.],
       [ 29.],
       [ 30.]])

In [14]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        a = dataset[i:(i + window_size), 0]
        dataX.append(a)
        dataY.append(dataset[i + window_size, 0])
    return np.array(dataX), np.array(dataY)

In [15]:
if use_LSTM:
    X_train, y_train = create_dataset(ts_train, lstm_num_timesteps)
    X_test, y_test = create_dataset(ts_test, lstm_num_timesteps)
else:
    X_train, y_train = create_dataset(ts_train, mlp_num_features)
    X_test, y_test = create_dataset(ts_test, mlp_num_features)
    
# the train and test matrices end up shorter than the respective timeseries by window_size + 1!
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((95, 5), (15, 5), (95,), (15,))

In [16]:
X_train[:5,:]

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 2.,  3.,  4.,  5.,  6.],
       [ 3.,  4.,  5.,  6.,  7.],
       [ 4.,  5.,  6.,  7.,  8.],
       [ 5.,  6.,  7.,  8.,  9.]])

In [17]:
y_train[:5]

array([  6.,   7.,   8.,   9.,  10.])

In [18]:
X_test[:5,:]

array([[ 21.,  22.,  23.,  24.,  25.],
       [ 22.,  23.,  24.,  25.,  26.],
       [ 23.,  24.,  25.,  26.,  27.],
       [ 24.,  25.,  26.,  27.,  28.],
       [ 25.,  26.,  27.,  28.,  29.]])

In [19]:
y_test[:5]

array([ 26.,  27.,  28.,  29.,  30.])

In [20]:
if use_LSTM:
    # reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))

In [25]:
model = Sequential()

if use_LSTM:
    
    # the last state for each sample at index i in a batch will be used as initial state
    # for the sample of index i in the following batch
    if lstm_stateful:
        #
        #
        if lstm_stack_layers:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            model.add(LSTM(num_neurons,
                       stateful = True))
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer='adam')
            
        # 
        elif lstm_predict_sequences:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            model.add(TimeDistributedDense(lstm_num_predictions), activation = 'linear')  
            model.add(Activation("linear"))  
            model.compile(loss='mean_squared_error', optimizer='adam')
            
        #    
        else:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True))
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer='adam')
        
        for i in range(num_epochs):
            print('epoch: ' + str(i))
            # shuffle must be False!
            model.fit(X_train, y_train, nb_epoch = 1, batch_size = batch_size, shuffle = False)
            model.reset_states()
 

    # stateful == False    
    else:        
        
        if lstm_stack_layers:
            # input_dim: dimensionality of the input (alternatively, input_shape)
            # required when using this layer as the first layer in a model
            model.add(LSTM(num_neurons, input_dim = lstm_num_features, return_sequences = True))
            model.add(LSTM(num_neurons))
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)
        # 
        # 
        elif lstm_predict_sequences:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            model.add(TimeDistributedDense(lstm_num_predictions))  
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)
            
        else:
            model.add(LSTM(num_neurons, input_dim = lstm_num_features))
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)
        
   

# feedforward
else:
    
    model.add(Dense(num_neurons, input_dim = mlp_num_features, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)


/home/key/software/anaconda3/envs/tf3.5/lib/python3.5/site-packages/keras/layers/core.py:1112: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


Exception: Error when checking model target: expected timedistributeddense_3 to have 3 dimensions, but got array with shape (95, 1)

In [ ]:
test_loss = np.nan
if lstm_stateful:
    test_loss = model.evaluate(X_test, y_test, batch_size = batch_size)
else:
    test_loss = model.evaluate(X_test, y_test, batch_size = X_test.shape[0])
test_loss

In [ ]:
if lstm_stateful:
    model.reset_states()
    pred_train = model.predict(X_train, batch_size = batch_size)
    model.reset_states()
    pred_test = model.predict(X_test, batch_size = batch_size)
else:
    pred_train1 = model.predict(X_train, batch_size = X_train.shape[0])
    pred_test1 = model.predict(X_test, batch_size = X_test.shape[0])
    pred_train2 = model.predict(X_train, batch_size = batch_size)
    pred_test2 = model.predict(X_test, batch_size = batch_size)

In [ ]:
#if not lstm_stateful:
#    print(np.round(pred_test1) == np.round(pred_test2))

In [ ]:
if not lstm_stateful:
    pred_test = pred_test1
    pred_train = pred_train1

In [ ]:
for i in X_test:
    if lstm_stateful:
        model.reset_states()
    #print(i)
    r = i.reshape(1, len(i), 1)
    #print(i.shape), print(r.shape)
    print(model.predict(r))

In [ ]:
pred_test

In [ ]:
def calc_dependent_predictions(model, data, prediction_window):
    prediction_seqs = []
    for i in range(int(len(data)/prediction_window)):
        print('Calculating predictions starting from: {}'.format(i))
        curr_frame = data[i*prediction_window]
        predicted = []
        for j in range(prediction_window):
            #print('Calculating single prediction: {}'.format(j))
            #print(curr_frame)
            pred = model.predict(curr_frame[np.newaxis,:,:])[0,0]
            #pred = model.predict(curr_frame.reshape(1, len(curr_frame), 1)) # same
            #print(pred)
            predicted.append(pred)
            curr_frame = curr_frame[1:] 
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [ ]:
prediction_window = 5 

prediction_seqs_train = calc_dependent_predictions(model, X_train, prediction_window)
prediction_seqs_test = calc_dependent_predictions(model, X_test, prediction_window)

In [ ]:
prediction_seqs_train 

In [ ]:
prediction_seqs_test

In [ ]:
y_train[:10]

In [ ]:
pred_train[:10,0]

In [ ]:
y_test[:10]

In [ ]:
pred_test[:10,0]

In [ ]:
if scale:
    pred_train = scaler.inverse_transform(pred_train)
    y_train = scaler.inverse_transform(y_train.reshape(-1,1))
    pred_test = scaler.inverse_transform(pred_test)
    y_test = scaler.inverse_transform(y_test.reshape(-1,1))


In [ ]:
y_train[:10],pred_train[:10,0]

In [ ]:
y_test[:],pred_test[:,0]

In [ ]:
# calculate root mean squared error
rsme_train = math.sqrt(mean_squared_error(y_train, pred_train[:,0]))
print('Train Score: %.2f RMSE' % (rsme_train))
rsme_test = math.sqrt(mean_squared_error(y_test, pred_test[:,0]))
print('Test Score: %.2f RMSE' % (rsme_test))

In [ ]:
print(len(ts_train), len(pred_train), len(y_train))
len(ts_test), len(pred_test), len(y_test) 

In [ ]:
# shift train predictions for plotting
pred_train_shifted = np.empty_like(ts_all)
print(pred_train_shifted.size)
pred_train_shifted[:, :] = np.nan
# train predictions start at position window_size + 1 (or window_size, if counting from 0)
pred_train_shifted[window_size : len(pred_train) + window_size, :] = pred_train
pred_train_shifted[:13]

In [ ]:
# shift test predictions for plotting
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features
pred_test_shifted = np.empty_like(ts_all)
pred_test_shifted[:, :] = np.nan
pred_test_shifted[len(pred_train) + (window_size * 2) : len_overall + 1, :] = pred_test
pred_test_shifted[-13:]

In [ ]:
plt.plot(ts_all)
plt.plot(pred_train_shifted)
plt.plot(pred_test_shifted)
plt.savefig(testname + '_lstm_' + str(use_LSTM) + '_stateful_' + str(lstm_stateful) + '_window_' + str(window_size) +
            '_epochs_' + str(num_epochs) + '_2layers_' + str(lstm_stack_layers) + '_scale_' + str(scale) + '.png')
plt.show()

In [ ]:
plot_start = -30
plot_end = -1
plt.plot(ts_all[plot_start:plot_end])
plt.plot(pred_train_shifted[plot_start:plot_end])
plt.plot(pred_test_shifted[plot_start:plot_end])
plt.show()


In [ ]:
def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()

In [ ]:
plot_results(pred_train, y_train)

In [ ]:
plot_results(pred_test, y_test)

In [ ]:
def plot_results_multiple(predicted_data, true_data, prediction_window):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_window)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()

In [ ]:
plot_results_multiple(prediction_seqs_train, y_test, prediction_window)

In [ ]:
plot_results_multiple(prediction_seqs_test, y_test, prediction_window)

In [ ]:
#seq2seq
# predict and append prediction to existing values
# TimeDistributedDense after return_sequences True

In [ ]:
# https://groups.google.com/forum/#!topic/keras-users/9GsDwkSdqBg

In [ ]:
# https://github.com/fchollet/keras/issues/1029#issuecomment-160845826
from keras.models import Sequential  
from keras.layers.core import TimeDistributedDense, Activation, Dropout  
from keras.layers.recurrent import GRU
import numpy as np

def _load_data(data, steps = 40):  
    docX, docY = [], []
    for i in range(0, data.shape[0]/steps-1):
        docX.append(data[i*steps:(i+1)*steps,:])
        docY.append(data[(i*steps+1):((i+1)*steps+1),:])
    return np.array(docX), np.array(docY)

def train_test_split(data, test_size=0.15):  
    #    This just splits data to training and testing parts
    X,Y = _load_data(data)
    ntrn = round(X.shape[0] * (1 - test_size))
    perms = np.random.permutation(X.shape[0])
    X_train, Y_train = X.take(perms[0:ntrn],axis=0), Y.take(perms[0:ntrn],axis=0)
    X_test, Y_test = X.take(perms[ntrn:],axis=0),Y.take(perms[ntrn:],axis=0)
    return (X_train, Y_train), (X_test, Y_test) 

np.random.seed(0)  # For reproducability
data = np.genfromtxt('closingAdjLog.csv', delimiter=',')  # data = a TxN matrix
(X_train, y_train), (X_test, y_test) = train_test_split(np.flipud(data))  # retrieve data
print "Data loaded."

in_out_neurons = 20  
hidden_neurons = 200

model = Sequential()  
model.add(GRU(hidden_neurons, input_dim=in_out_neurons, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributedDense(in_out_neurons))  
model.add(Activation("linear"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop") 
print "Model compiled."

# and now train the model. 
model.fit(X_train, y_train, batch_size=30, nb_epoch=200, validation_split=0.1)  
predicted = model.predict(X_test)  
print np.sqrt(((predicted - y_test) ** 2).mean(axis=0)).mean()  # Printing RMSE 

@joetigger
joetigger commented on Nov 20, 2015

@M-Taha Thanks for the example. According to you, TimeDistributedDense is for saving time/memory. Therefore it should work if I set in_out_neurons to 1. However, according to @EdwardRaff it is for constraining many outputs to use the same function. I'm confused.. Which one is correct?
@viveksck
viveksck commented on Nov 20, 2015

@placebokkk : Thanks for the sample code. I tried it, but it seems that it is not able to even memorize the training data.

from keras.models import Sequential  
from keras.layers.core import TimeDistributedDense, Activation, Dropout  
from keras.layers.recurrent import GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
import numpy as np
maxlen = 2

batch_size = 1
nb_word = 4
nb_tag = 2

X_train = [[1,2],[1,3]]#two sequences, one is [1,2] and another is [1,3]
Y_train = [[[0,1],[1,0]],[[0,1],[1,0]]] #the output should be 3D and one-hot for softmax output with categorical_crossentropy
X_test = [[1,2],[1,3]]
Y_test = [[[0,1],[1,0]],[[0,1],[1,0]]]

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

Y_train = np.asarray(Y_train, dtype='int32')
Y_test = np.asarray(Y_test, dtype='int32')

model = Sequential()
model.add(Embedding(nb_word, 128))
model.add(LSTM(128, return_sequences=True))
model.add(TimeDistributedDense(nb_tag))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms)

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=100, show_accuracy=True)
res = model.predict_classes(X_test)
print('res',res)